In [2]:
%matplotlib widget

In [3]:
import pandas as pd
import numpy as np
from scipy.signal import butter, sosfiltfilt
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pathlib import Path
import lightgbm as lgb

from sklearn.metrics import f1_score

from PfyMU.gait.train_classifier.core import load_datasets
from PfyMU.features import *

plt.style.use('ggplot')

# Setup

In [4]:
def mag_band_filter(x, fs):
    sos = butter(
        1, 
        [2 * 0.25 / fs, 2 * 5 / fs], 
        btype='band', 
        output='sos'
    )
    
    return sosfiltfilt(sos, np.linalg.norm(x, axis=1))

steps = {
    'walking': 0.4,
    'walking-impaired': 0.2,
    'sitting': 900,
    'standing': 300,
    'stairs-ascending': 0.3,
    'stairs-descending': 0.3,
    'cycling-50W': 0.3,
    'cycling-100W': 0.3,
    'default': 1.0
}

# Load the Data

In [5]:
base_path = Path('/home/lukasadamowicz/Documents/Datasets/processed')

datasets = [
    base_path / 'bluesky2',
    base_path / 'daliac',
    base_path / 'ltmm',
    base_path / 'usc-had'
]

X, Y, subjects, activities = load_datasets(
    paths=datasets,
    goal_fs=50.0,
    window_length=3.0,
    window_step=steps,
    acc_mag=False,
    signal_function=mag_band_filter
)

Y2 = Y.copy()
Y2[['stair' in i for i in activities]] = 1

# Randomize validation/test splits

In [6]:
random.seed(5)
rnd_subjects = [i for i in np.unique(subjects) if np.unique(activities[subjects==i]).size > 3]
random.shuffle(rnd_subjects)

training_masks, validation_masks, testing_masks = [], [], []

for i in range(0, len(rnd_subjects), 4):
    trm = np.ones(len(subjects), dtype='bool')
    vm = np.zeros_like(trm, dtype='bool')
    tem = np.zeros_like(trm, dtype='bool')
    
    for j in range(4):
        trm &= subjects != rnd_subjects[i + j]
        if j < 2:
            vm |= subjects == rnd_subjects[i + j]
        else:
            tem |= subjects == rnd_subjects[i + j]
    
    training_masks.append(trm)
    validation_masks.append(vm)
    testing_masks.append(tem)

# Feature Computation

In [7]:
FB = Bank(window_length=None, window_step=None)

FB.load('final_features.json')

In [8]:
X_feat, fnames = FB.compute(X, fs=50.0, windowed=True, columns=[''])

# LightGBM model setup

In [10]:
best_params = {}
with open('gait_classifier/lgb_params.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        parts = line.strip('\n').split(': ')
        
        best_params[parts[0]] = float(parts[1]) if '.' in parts[1] else int(parts[1])

# Cross validation

In [21]:
thresh = 0.71

In [22]:
f1, f1_2 = [], []
tp, tp2 = [], []
fp, fp2 = [], []

print('Validation set performance')
for trm, vm, tem in zip(training_masks, validation_masks, testing_masks):
    lgb_cls = lgb.LGBMClassifier(n_estimators=125, **best_params)
    lgb_cls2 = lgb.LGBMClassifier(n_estimators=125, **best_params)
    
    lgb_cls.fit(X_feat[trm], Y[trm])
    lgb_cls2.fit(X_feat[trm], Y2[trm])
    
    y_pred = lgb_cls.predict_proba(X_feat[vm])[:, 1]
    y2_pred = lgb_cls2.predict_proba(X_feat[vm])[:, 1]
    
    # compute metrics
    f1.append(f1_score(Y[vm], y_pred > thresh))
    f1_2.append(f1_score(Y2[vm], y2_pred > thresh))
    tp.append((Y[vm] & (y_pred > thresh)).sum() / Y[vm].sum())
    tp2.append((Y2[vm] & (y2_pred > thresh)).sum() / Y2[vm].sum())
    fp.append((~Y[vm].astype(bool) & (y_pred > thresh)).sum() / (Y[vm].size - Y[vm].sum()))
    fp2.append((~Y2[vm].astype(bool) & (y2_pred > thresh)).sum() / (Y2[vm].size - Y2[vm].sum()))
    
    print(f'F1: {f1[-1]*100:6.1f}{f1_2[-1]*100:10.1f}', end='')
    print(f'  TP: {tp[-1]*100:6.1f}{tp2[-1]*100:10.1f}', end='')
    print(f'  FP: {fp[-1]*100:6.1f}{fp2[-1]*100:10.1f}')
    
print('\n', '-' * 50)
print(f'Mean (SD) F1: {np.mean(f1)*100:.1f}({np.std(f1)*100:.1f})   {np.mean(f1_2)*100:.1f}({np.std(f1_2)*100:.1f})')
print(f'Mean (SD) TP: {np.mean(tp)*100:.1f}({np.std(tp)*100:.1f})   {np.mean(tp2)*100:.1f}({np.std(tp2)*100:.1f})')
print(f'Mean (SD) FP: {np.mean(fp)*100:.1f}({np.std(fp)*100:.1f})   {np.mean(fp2)*100:.1f}({np.std(fp2)*100:.1f})')

Validation set performance
F1:   95.3      99.5  TP:   93.7      99.7  FP:    2.1       0.6
F1:   92.6      97.7  TP:   95.9     100.0  FP:    4.1       2.9
F1:   93.2      96.3  TP:   93.4      96.4  FP:    3.1       2.4
F1:   92.0      97.4  TP:   97.8     100.0  FP:    5.2       3.3
F1:   82.0      80.2  TP:   70.1      67.6  FP:    2.4       2.1
F1:   94.9      97.8  TP:   90.4      96.4  FP:    0.3       1.3
F1:   87.8      97.7  TP:   79.3      95.6  FP:    0.9       0.4
F1:   94.3      99.0  TP:   95.3      98.2  FP:    7.0       0.4
F1:   94.4      97.1  TP:   97.0      98.0  FP:    4.8       3.6
F1:   93.5      98.8  TP:   94.4      98.4  FP:    3.4       1.7
F1:   79.5      95.5  TP:   74.3     100.0  FP:    5.5       6.9

 --------------------------------------------------
Mean (SD) F1: 90.9(5.2)   96.1(5.1)
Mean (SD) TP: 89.2(9.4)   95.5(8.9)
Mean (SD) FP: 3.5(1.9)   2.3(1.8)
